In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [5]:
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'

In [6]:
torch.cuda.set_device(device)

In [7]:
from utils import load_cifar10, load_cifar100, load_tinyimagenet

In [8]:
# load cifar10 data
trainloader, testloader = load_cifar10(data_dir="../data/cifar10", batch_size=128, 
                                       test_batch = 128,train_shuffle=True)
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
from utils import train_valid_split

In [10]:
# load dataloader for steal
stealloader,st_testloader = load_cifar100(data_dir="../data/cifar100", batch_size=128, train_shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [11]:
st_trloader, st_valloader = train_valid_split(stealloader,10000)

total data: 10000


In [12]:
# load tinyimagenet for outlier exposure
outlier_loader = load_tinyimagenet(data_dir="../data/tiny-imagenet-200",
                                   batch_size=128, train_shuffle=True)

In [58]:
class args:
    save_dir = "../results"
    orig_model = "cifar_orig_logit.pth"
#     fake_model = "cifar_fake_random_net.pth"
    fake_model = "cifar_fake_logit_ML.pth"
    fake_model_swd = "cifar_fake_logit_ML_swd.pth"
    tau = 0.5
#     nb_stolen = 10000 # attack model 학습시 사용할 데이터 수
#     st_trloader = "attnet_trloader.dl"
#     st_valloader = "attnet_valloader.dl"
#     att_epochs = 300
    att_model = "cifar_attacknet_logit_ml_tau%.1f.pth"%(tau)
    att_model_swd = "cifar_attacknet_logit_ml_swd_tau%.1f.pth"%(tau)

In [59]:
args.att_model

'cifar_attacknet_logit_ml_tau0.5.pth'

In [60]:
from cifar_models import Net, Net_logit, AttackNet
from utils import CombNet, CombNet_logit, Net_softmax

In [61]:
import os
net = AttackNet()
net.load_state_dict(torch.load(os.path.join(args.save_dir,args.orig_model),map_location='cpu'))
net = Net_softmax(net)

net_fake = AttackNet()
net_fake.load_state_dict(torch.load(os.path.join(args.save_dir,args.fake_model),map_location='cpu'))
net_fake = Net_softmax(net_fake)

net_fake_swd = AttackNet()
net_fake_swd.load_state_dict(torch.load(os.path.join(args.save_dir,args.fake_model_swd),map_location='cpu'))
net_fake_swd = Net_softmax(net_fake_swd)


comb_net = CombNet(net, net_fake, args.tau)
comb_net_swd = CombNet(net, net_fake_swd, args.tau)

In [57]:
print(os.path.join(args.save_dir, args.att_model))
print(os.path.join(args.save_dir, args.att_model_swd))

../results/cifar_attacknet_logit_ml_tau0.5.pth
../results/cifar_attacknet_logit_ml_swd_tau0.5.pth


In [33]:
criterion = nn.CrossEntropyLoss()
criterion_NLL = nn.NLLLoss()

# Adversarial PGD attacks

In [46]:
att_net = AttackNet()
att_net.load_state_dict(torch.load(os.path.join(args.save_dir, args.att_model),map_location='cpu'))

att_net_swd = AttackNet()
att_net_swd.load_state_dict(torch.load(os.path.join(args.save_dir, args.att_model_swd),map_location='cpu'))

<All keys matched successfully>

In [21]:
from torchattacks import PGD

In [47]:
atk_logit_ml = PGD(att_net, eps=8/255, alpha=2/255, steps=7)
atk_logit_ml.save(data_loader=testloader,save_path = os.path.join(args.save_dir, 'cifar_adv_logit_ml0.5.pth'))

- Save progress: 100.00 % / Robust accuracy: 2.70 % / L2: 19.18600 (0.097 it/s) 	


In [48]:
atk_logit_ml_swd = PGD(att_net_swd, eps=8/255, alpha=2/255, steps=7)
atk_logit_ml_swd.save(data_loader=testloader,save_path = os.path.join(args.save_dir, 'cifar_adv_logit_ml_swd0.5.pth'))

- Save progress: 100.00 % / Robust accuracy: 3.31 % / L2: 19.22256 (0.102 it/s) 	


In [49]:
adv_images, adv_labels = torch.load(os.path.join(args.save_dir, 'cifar_adv_logit_ml0.5.pth'))
adv_images_swd, adv_labels_swd = torch.load(os.path.join(args.save_dir, 'cifar_adv_logit_ml_swd0.5.pth'))

In [50]:
from torch.utils.data import TensorDataset, DataLoader
adv_loader = DataLoader(TensorDataset(adv_images.float(),adv_labels),batch_size=128, shuffle=False)
adv_loader_swd = DataLoader(TensorDataset(adv_images_swd.float(),adv_labels_swd),batch_size=128, shuffle=False)

In [51]:
from utils import test_model

In [52]:
att_net.to(device)
print(test_model(att_net,adv_loader,criterion,device,100.0))
att_net.cpu()
comb_net.to(device)
print(test_model(comb_net,adv_loader,criterion_NLL,device,100.0,pred_prob=True))

(tensor(3.7704), tensor(2.7000))
(tensor(5.1928), tensor(26.1800))


In [53]:
comb_net.cpu()

CombNet(
  (net_orig): Net_softmax(
    (model): AttackNet(
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (classifier): Sequential(
        (0): Linear(in_features=2048, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=10, bias=True)
      )
    )
  )
  (net_fake): Net_softmax(
    (model): AttackNet(
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (classifier): Sequential(
        (0)

In [54]:
att_net_swd.to(device)
print(test_model(att_net_swd,adv_loader_swd,criterion,device,100.0))
att_net_swd.cpu()
comb_net_swd.to(device)
print(test_model(comb_net_swd,adv_loader_swd,criterion_NLL,device,100.0,pred_prob=True))

(tensor(3.9693), tensor(3.3100))
(tensor(6.7935), tensor(28.4400))


In [55]:
comb_net_swd.cpu()

CombNet(
  (net_orig): Net_softmax(
    (model): AttackNet(
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (classifier): Sequential(
        (0): Linear(in_features=2048, out_features=256, bias=True)
        (1): Linear(in_features=256, out_features=10, bias=True)
      )
    )
  )
  (net_fake): Net_softmax(
    (model): AttackNet(
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (classifier): Sequential(
        (0)